In [25]:
import string
import unidecode

import nltk
from collections import defaultdict
#from nltk.stem.snowball import EnglishStemmer
from nltk.stem import SnowballStemmer #Stemming: Porter2 >  more aggressive than the Porter stemmer, Slightly faster, 
from nltk.tokenize import word_tokenize
#from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import FreqDist
%matplotlib inline


In [13]:
document_1 = "I love watching movies when it's cold outside"
document_2 = "Toy Story is the best animation movie ever, I love it!"
document_3 = "Watching horror movies alone at night is really scary"
document_4 = "He loves to watch films filled with suspense and unexpected plot twists"
document_5 = "My mom loves to watch movies. My dad hates movie theaters. My brothers like any kind of movie. And I haven't watched a single movie since I got into college"

documents = [document_1, document_2, document_3, document_4, document_5]
documents[0]

"I love watching movies when it's cold outside"

In [14]:
def clean_text (text):
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove todas as pontuações: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text = text.replace('\n',' ').strip() 
    text = text.lower()
    text = unidecode.unidecode(text)
    return text

clean_text(documents[0])

'i love watching movies when its cold outside'

In [17]:
#redução das palavras para sua raiz (stemming), remoção de stopwords e palavras com menos de 2 caracteres, e criação do vocabulário com a quantidade de ocorrência de cada palavra em todos os documentos
def vocabulary_stemming(documents):
    docs_stem_words = []
    vocabulary = {}
    stop_words = stopwords.words('english')
    snowball_stemmer = SnowballStemmer("english")

    for index in range(len(documents)):
        text = documents[index]
        tokentext = word_tokenize(clean_text(text))
        stem_words  = [snowball_stemmer.stem(word) for word in tokentext if not word in stop_words and len(word) > 2 and word not in string.punctuation]
        docs_stem_words.append(stem_words)

        #Inicializa vocabulário sem repetição de palavras
        for word in stem_words:
            vocabulary[word] = 0

    #Contabiliza ocorrência de cada palavra em todos os documentos
    for words in docs_stem_words:
        for word in words:
            vocabulary[word] += 1
    
    return vocabulary, docs_stem_words

vocabulary, docs_stem_words = vocabulary_stemming(documents)

In [ ]:
#Outra opção de tratamento na construção do vocabulário e frequencias
snowball_stemmer = SnowballStemmer("english")
stop_words = stopwords.words('english')

tokens = sum([word_tokenize(clean_text(document)) for document in documents], [])
stem_words  = [snowball_stemmer.stem(word) for word in tokens if not word in stop_words and len(word) > 2 and word not in string.punctuation]
words_frequency2 = FreqDist(stem_words)
words_frequency2.plot(30, cumulative = False)
words_frequency1 = FreqDist(tokens)
words_frequency1.plot(30, cumulative = False)

In [20]:
docs_stem_words

[['love', 'watch', 'movi', 'cold', 'outsid'],
 ['toy', 'stori', 'best', 'anim', 'movi', 'ever', 'love'],
 ['watch', 'horror', 'movi', 'alon', 'night', 'realli', 'scari'],
 ['love', 'watch', 'film', 'fill', 'suspens', 'unexpect', 'plot', 'twist'],
 ['mom',
  'love',
  'watch',
  'movi',
  'dad',
  'hate',
  'movi',
  'theater',
  'brother',
  'like',
  'kind',
  'movi',
  'havent',
  'watch',
  'singl',
  'movi',
  'sinc',
  'got',
  'colleg']]

In [19]:
vocabulary.items()

dict_items([('love', 4), ('watch', 5), ('movi', 7), ('cold', 1), ('outsid', 1), ('toy', 1), ('stori', 1), ('best', 1), ('anim', 1), ('ever', 1), ('horror', 1), ('alon', 1), ('night', 1), ('realli', 1), ('scari', 1), ('film', 1), ('fill', 1), ('suspens', 1), ('unexpect', 1), ('plot', 1), ('twist', 1), ('mom', 1), ('dad', 1), ('hate', 1), ('theater', 1), ('brother', 1), ('like', 1), ('kind', 1), ('havent', 1), ('singl', 1), ('sinc', 1), ('got', 1), ('colleg', 1)])

In [22]:
#Criar índice invertido
def invertedIndex(vocabulary, docs_stem_words): 
    invertedList = dict()
    for term in vocabulary:
        invertedList[term] = list()
        index = 0
        for stem_words in docs_stem_words:
            frequencia = 0
            for word in stem_words:
                if word == term:
                    frequencia += 1
            if frequencia > 0:
                invertedList[term].append([index, frequencia])
            index += 1
    return invertedList

invertedList = invertedIndex(vocabulary, docs_stem_words)
invertedList.items()

dict_items([('love', [[0, 1], [1, 1], [3, 1], [4, 1]]), ('watch', [[0, 1], [2, 1], [3, 1], [4, 2]]), ('movi', [[0, 1], [1, 1], [2, 1], [4, 4]]), ('cold', [[0, 1]]), ('outsid', [[0, 1]]), ('toy', [[1, 1]]), ('stori', [[1, 1]]), ('best', [[1, 1]]), ('anim', [[1, 1]]), ('ever', [[1, 1]]), ('horror', [[2, 1]]), ('alon', [[2, 1]]), ('night', [[2, 1]]), ('realli', [[2, 1]]), ('scari', [[2, 1]]), ('film', [[3, 1]]), ('fill', [[3, 1]]), ('suspens', [[3, 1]]), ('unexpect', [[3, 1]]), ('plot', [[3, 1]]), ('twist', [[3, 1]]), ('mom', [[4, 1]]), ('dad', [[4, 1]]), ('hate', [[4, 1]]), ('theater', [[4, 1]]), ('brother', [[4, 1]]), ('like', [[4, 1]]), ('kind', [[4, 1]]), ('havent', [[4, 1]]), ('singl', [[4, 1]]), ('sinc', [[4, 1]]), ('got', [[4, 1]]), ('colleg', [[4, 1]])])

In [31]:
#Exibe os dados formatados
fmt = '{:<8}{:<15}{:<8}{}'

print(fmt.format('', 'Vocabulary', 'ni', 'Inverted Lists'))
for i, (vocab, ni, ocurrencies) in enumerate(zip(vocabulary.keys(), vocabulary.values(), invertedList.values())):
    print(fmt.format(i, vocab, ni, ocurrencies))

Vocabulary     ni      Inverted Lists
0       love           4       [[0, 1], [1, 1], [3, 1], [4, 1]]
1       watch          5       [[0, 1], [2, 1], [3, 1], [4, 2]]
2       movi           7       [[0, 1], [1, 1], [2, 1], [4, 4]]
3       cold           1       [[0, 1]]
4       outsid         1       [[0, 1]]
5       toy            1       [[1, 1]]
6       stori          1       [[1, 1]]
7       best           1       [[1, 1]]
8       anim           1       [[1, 1]]
9       ever           1       [[1, 1]]
10      horror         1       [[2, 1]]
11      alon           1       [[2, 1]]
12      night          1       [[2, 1]]
13      realli         1       [[2, 1]]
14      scari          1       [[2, 1]]
15      film           1       [[3, 1]]
16      fill           1       [[3, 1]]
17      suspens        1       [[3, 1]]
18      unexpect       1       [[3, 1]]
19      plot           1       [[3, 1]]
20      twist          1       [[3, 1]]
21      mom            1       [[4, 1]]
22      da

# Realizando Busca

In [44]:
def lookup_query(self, query):
    docs_index = [self[term] for term in query.split(' ') if term in self.keys()]
    return docs_index

In [47]:
lookup_query(invertedList,'watch with love')

[[[0, 1], [2, 1], [3, 1], [4, 2]], [[0, 1], [1, 1], [3, 1], [4, 1]]]

dict_keys(['love', 'watch', 'movi', 'cold', 'outsid', 'toy', 'stori', 'best', 'anim', 'ever', 'horror', 'alon', 'night', 'realli', 'scari', 'film', 'fill', 'suspens', 'unexpect', 'plot', 'twist', 'mom', 'dad', 'hate', 'theater', 'brother', 'like', 'kind', 'havent', 'singl', 'sinc', 'got', 'colleg'])